### Design a Python Method to Obtain the Latest Constructor Standings

To get the latest standings in the season.

In [1]:
import pandas as pd
import requests

In [2]:
# Latest data url:
url = 'http://ergast.com/api/f1/current/last/constructorStandings.json'

In [3]:
# Request.
r = requests.get(url)
r.raise_for_status()
data = r.json()['MRData']['StandingsTable']['StandingsLists'][0]

In [28]:
# Store the data from the request in variables.
season_no = data['season']
round_no = data['round']
constructor_standings = data['ConstructorStandings']

In [23]:
# Function to return value with a preceding 0 if required.
def p_z(value):
    s_val = str(value)
    return f"0{s_val}" if len(s_val) == 1 else s_val

In [29]:
# Manipulate the data to turn it into a more usable format.
df = pd.DataFrame(constructor_standings)
df['Constructor Name'] = df['Constructor'].apply(lambda value: value['name'])
df['Constructor URL'] = df['Constructor'].apply(lambda value: value['url'])
df['Constructor Nationality'] = df['Constructor'].apply(lambda value: value['nationality'])
df['Season'] = season_no
df['Round'] = round_no

# Create unique identifier.
df['UID'] = df['position'].apply(lambda value: str(season_no) + p_z(round_no) + p_z(value))

df = df.set_index('UID').drop(columns=['positionText', 'Constructor'])

df.columns = ['Position', 'Points', 'Wins', 'Constructor Name', 'Constructor URL',
                  'Constructor Nationality', 'Season Year', 'Round Number']
df = df[['Position', 'Constructor Name', 'Points', 'Wins', 'Constructor URL',
                 'Constructor Nationality', 'Season Year', 'Round Number']]
df

,Position,Constructor Name,Points,Wins,Constructor URL,Constructor Nationality,Season Year,Round Number
UID,,,,,,,,
20211901,1,Mercedes,521.5,7,http://en.wikipedia.org/wiki/Mercedes-Benz_in_...,German,2021,19
20211902,2,Red Bull,510.5,10,http://en.wikipedia.org/wiki/Red_Bull_Racing,Austrian,2021,19
20211903,3,Ferrari,287.5,0,http://en.wikipedia.org/wiki/Scuderia_Ferrari,Italian,2021,19
20211904,4,McLaren,256,1,http://en.wikipedia.org/wiki/McLaren,British,2021,19
20211905,5,Alpine F1 Team,112,1,http://en.wikipedia.org/wiki/Alpine_F1_Team,French,2021,19
20211906,6,AlphaTauri,112,0,http://en.wikipedia.org/wiki/Scuderia_AlphaTauri,Italian,2021,19
20211907,7,Aston Martin,68,0,http://en.wikipedia.org/wiki/Aston_Martin_in_F...,British,2021,19
20211908,8,Williams,23,0,http://en.wikipedia.org/wiki/Williams_Grand_Pr...,British,2021,19
20211909,9,Alfa Romeo,11,0,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Swiss,2021,19


In [6]:
# Create a filename.
filename = f"{season_no}_r{p_z(round_no)}_Constructor_Standings.csv"

In [7]:
# Save the data to the filename.
df.to_csv(f"../api_data/transformed_data/{filename}")